# Initialize environment

In [ ]:
from classes import *
import pandas as pd
from datetime import datetime, timedelta, time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
from choo_raptor import ChochocrewAlgorithm

%load_ext sparkmagic.magics
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install pyarrow
!pip install fastparquet
!git lfs pull

# Load data and instantiate python classes 

In [ ]:
from load_data import get_data

# It can takes a few minutes
stops, routes, footpaths, stops_df =  get_data()

In [ ]:
stops_save = stops.copy()
routes_save = routes.copy()
footpaths_save = footpaths.copy()

## Chocho_Raptor
There are some tests that you can check

In [ ]:
from tests import *

# Some predefined itineraries

# test1(stops_save.copy(), routes_save.copy(), footpaths_save.copy())
# test2(stops_save.copy(), routes_save.copy(), footpaths_save.copy())
# test3(stops_save.copy(), routes_save.copy(), footpaths_save.copy())
# test4(stops_save.copy(), routes_save.copy(), footpaths_save.copy())

## Interface
You can run the cell below to choose your journey

In [ ]:
# Run this cell and choose your itinerary
from interface import process_journey, plot_journey, plot_trip
stops_name_interface = []

for s in stops : 
    stops_name_interface.append(s.stop_name)
    
@interact(dep_stop=widgets.Dropdown(
                options=sorted(stops_name_interface),
                description='Departure stop:', 
                style=dict(description_width='initial'),
                  value = "Zürich Flughafen"), 
          arr_stop=widgets.Dropdown(
                options=sorted(stops_name_interface),
                description='Arrival stop:', 
                style=dict(description_width='initial'),
                  value = "Dietikon"), 
          arr_time=widgets.TimePicker(
                description='Max arrival time',
                style=dict(description_width='initial')), 
          prob=widgets.FloatSlider(
                value = 1.0,
                min=0,
                max=1.0,
                step=0.01,
                disabled=False,
                continuous_update=False,
                description='Probability:'))

def everything(dep_stop = None, arr_stop = None, arr_time= None, prob=None):

    if not (dep_stop == None and arr_stop == None and arr_time == None):
        if arr_time == None or prob == 1 or (dep_stop == arr_stop):
            if (dep_stop == arr_stop):
                print("Please choose a departure stop different from the arrival stop")
            elif arr_time == None:
                print("Please give an arrival time")
            elif prob == 1 :
                print("Please give a probability smaller than 1")
        else:

            print("Computing the journey...")
            
            dep_stop_algo = next((s for s in stops if  s.stop_name == dep_stop), None)
            arr_stop_algo = next((s for s in stops if s.stop_name == arr_stop), None)
            if not (isinstance(arr_time, str)):
                arr_time = arr_time.strftime("%H:%M:%S")

            arr_time = datetime.combine(datetime.today().date(), time.fromisoformat(arr_time))

            choo_algo = ChochocrewAlgorithm(starting_stop = dep_stop_algo,
                                        arrival_stop = arr_stop_algo, 
                                        prob_threshold = prob,
                                        max_arrival_time = arr_time, 
                                        min_arrival_time = arr_time,
                                        stops = stops_save.copy(),
                                        routes = routes_save.copy(), 
                                        footpaths = footpaths_save.copy(),
                                        verbose = False,
                                        with_proba = True)
            for stop in stops :
                stop.label_bags = [LabelBag(stop)]

            journeys = choo_algo.run()

            if len(journeys) == 0:
                print("No routes has been found")
            else:
                for index, journey in enumerate(journeys):
                        plot_journey(journey, index)
                        lon, lat, names = process_journey(journey['journey'],stops_df )
                        plot_trip(lon, lat, "trip" + str (index), names, stops_df)
                
everything()